In [1]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

In [2]:
data_df = pd.read_parquet("../dataset/train_df_all_english.parquet")
data_df = data_df[data_df["language"] == "English"]

In [4]:
subset_df = data_df.sample(100)

In [8]:
def generate_gpt_label(sentence):
        
    template = f"""Given the following news, output 0 if the ESG impact duration is below 2 years, 1 if the ESG impact duration is between 2 and 5 year and 2 if the ESG impact duration is more than 5 years. You only need to output the number, and do not need any further explanation.
    
    {sentence}"""
    
    conversation = [
            {"role": "system", "content": "You are an ESG analyst, skilled assessing the level and duration an event in the news article might have on the company"},
            {"role": "user", "content": template}
        ]
    
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=conversation,
        temperature=0.7
    )
    
    message = completion.choices[0].message.content
    
    return int(message)


subset_df["gpt_label"] = subset_df["sentence"].apply(generate_gpt_label)

In [10]:
len(subset_df[subset_df["gpt_label"] == subset_df["impact_length_idx"]])

49

In [13]:
subset_df.groupby(["gpt_label", "impact_length_idx"])["gpt_label"].count().unstack()

impact_length_idx,0.0,1.0,2.0
gpt_label,,,
0,9,9,7
1,7,13,13
2,4,11,27
